In [2]:
library(mimi)
library(missMDA)
library(reticulate)
library(ROCR)
np <- import("numpy")
library(parallel)

In [3]:
mca.logi.fn <- function(Y, R, ncp=4, maxit=100){
    Y[R==0] <- NA
    Y <- as.data.frame(Y)
    nC <- dim(Y)[2]
    for (i in 1:nC){
        Y[, i] <- as.factor(Y[, i])
    }
    res <- imputeMCA(Y, ncp=ncp, maxit=maxit, method="EM")
    return(res)
}

post.mca.fn <- function(res){
    probs <- res$tab.disj
    probs[probs<=0] = 0
    probs[probs>=1] = 1
    idxs <- seq(2, dim(probs)[2], 2)
}

In [4]:
mimi.logi.fn <- function(Y, X, R, CT, Cb, alp.init, theta.init, trace.it=T, maxit=100){
  p <- ncol(X)
  m <- dim(Y)[1]
  n <- dim(Y)[2]
  var.type <- rep("binomial", n)
  Y.miss <- Y
  Y.miss[R==0] <- NA
  res <- mimi(Y.miss, model="c", x=X, var.type=var.type, lambda1=CT, lambda2 = Cb, 
              alpha0=alp.init, theta0=theta.init, algo="bc", maxit=maxit, trace.it = trace.it, max.rank=10)
  return(res)
}

post.mimi.fn <- function(res){
    probs <-  exp(res$param)/(1+exp(res$param))
    return(probs)
}

In [5]:
probs2auc <- function(probs, Ynp, rYnp, Rnp){
    idxMat <- rYnp != -1 & Rnp == 0
    obs <- Ynp[idxMat]
    eProbs <- probs[idxMat]
    pred <- prediction(eProbs, obs)
    auc.res <- performance(pred, "auc")
    return(auc.res@y.values)
}

In [6]:
aucs2Idxs <- function(aucs){
    Cb.idx <- ifelse(which.max(aucs) %% 3==0, which.max(aucs) %/% 3, which.max(aucs) %/% 3 + 1)
    CT.idx <- ifelse(which.max(aucs) %% 3==0, 3, which.max(aucs) %% 3)
    return(c(Cb.idx, CT.idx))
}

In [7]:
Ynp <- np$load("./npData/Y.npz")
rYnp <- np$load("./npData/rY.npz")
Xnp <- np$load("./npData/X.npz")

In [8]:
m = dim(Xnp)[1]
n = dim(Xnp)[2]
p = dim(Xnp)[3]
alp.init <- rep(0, p)
theta.init <- matrix(rnorm(n*m), nrow=m) + 0.1
matX <- matrix(Xnp, nrow=m*n)

In [9]:
OR <- 50
Rnp.ps <- paste0("./npData/R_", OR, "_", 1:20, ".npz")
Rnp0 <- np$load(Rnp.ps[1])
CTs <- c(1, 1e2, 1e3)
Cbs <- c(1, 1e2, 1e3)

In [10]:
CbTs <- list()
CbTs[[1]] <- c(1, 1)
CbTs[[2]] <- c(1, 1e2)
CbTs[[3]] <- c(1, 1e3)
CbTs[[4]] <- c(1e2, 1)
CbTs[[5]] <- c(1e2, 1e2)
CbTs[[6]] <- c(1e2, 1e3)
CbTs[[7]] <- c(1e3, 1)
CbTs[[8]] <- c(1e3, 1e2)
CbTs[[9]] <- c(1e3, 1e3)

In [12]:
alp.init <- rep(0, p)
theta.init <- matrix(rnorm(n*m), nrow=m) + 0.1
runfn.CbT <- function(idx){
    print(idx)
    Cb <- CbTs[[idx]][1]
    CT <- CbTs[[idx]][2]
    res <- mimi.logi.fn(Ynp, matX, Rnp0, CT=CT, Cb=Cb, alp.init=alp.init, theta.init=theta.init)
    probs <- post.mimi.fn(res)
    cur.auc <- probs2auc(probs, Ynp, rYnp, Rnp0) 
    res <- list()
    res$v <- cur.auc
    res$CbT <- CbTs[[idx]]
    res
}

In [15]:
res.aucs <- mclapply(1:9, runfn.CbT, mc.cores=5)

In [18]:
selCb <- 1
selCT <- 1

In [19]:
runfn <- function(idx){
    Rnp.p <- Rnp.ps[idx]
    print(Rnp.p)
    Rnp <- np$load(Rnp.p)
    alp.init <- rep(0, p)
    theta.init <- matrix(rnorm(n*m), nrow=m) + 0.1
    res <- mimi.logi.fn(Ynp, matX, Rnp, CT=selCT, Cb=selCb, alp.init=alp.init, theta.init=theta.init)
    probs <- post.mimi.fn(res)
    cur.auc <- probs2auc(probs, Ynp, rYnp, Rnp) 
    res <- list()
    res$v <- cur.auc
    res$idx <- idx
    res
}

In [20]:
test.res.aucs <- mclapply(1:5, runfn, mc.cores=5)

In [21]:
test.res.aucs

[[1]]
[[1]]$v
[[1]]$v[[1]]
[1] 0.5796411


[[1]]$idx
[1] 1


[[2]]
[[2]]$v
[[2]]$v[[1]]
[1] 0.5782094


[[2]]$idx
[1] 2


[[3]]
[[3]]$v
[[3]]$v[[1]]
[1] 0.5764167


[[3]]$idx
[1] 3


[[4]]
[[4]]$v
[[4]]$v[[1]]
[1] 0.579491


[[4]]$idx
[1] 4


[[5]]
[[5]]$v
[[5]]$v[[1]]
[1] 0.5814661


[[5]]$idx
[1] 5

In [30]:
argidx <- which.max(sapply(res.aucs, function(x)x$v))
CbT <- res.aucs[[argidx]]$CbT

In [31]:
CbT

[1] 1 1